In [1]:

import pandas as pd
import numpy as np
from scipy.stats import ttest_ind
from sklearn.linear_model import LassoCV, RidgeCV
from sklearn.feature_selection import SequentialFeatureSelector
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.ensemble import GradientBoostingClassifier
from tqdm.auto import tqdm
df = pd.read_csv("하이닉스 power1_data.csv", thousands=',', encoding='utf-8')
# 등락률을 기준으로 다음 날 등락률 계산 후 target 생성
df['next_day_return'] = (df['종가'].shift(-1) - df['종가'])/ df['종가'] *100
df['target'] = df['next_day_return'].apply(lambda x: 2 if x > 0.25 else 1)
df.dropna(inplace=True)  # 마지막 행 삭제

first_column_name = df.columns[0]

#df=df.iloc[-300:,:]
X = df.drop(['날짜','target','next_day_return'], axis=1) 
y = df['target']


# Random Forest for feature importance
rf = RandomForestRegressor()
rf.fit(X, y)

features_rf_sorted = sorted(zip(X.columns, rf.feature_importances_), key=lambda x: x[1], reverse=True)
features_rf_df = pd.DataFrame(features_rf_sorted, columns=['Feature', 'RF Importance'])

# 상관관계가 가장 높은 상위 8개의 피처를 선정
top_7_features = [feature for feature, importance in features_rf_sorted[:5]]

# X와 y 정의 (df2와 이전 코드에서의 처리 과정을 바탕으로)dp
X_top8 = X[top_7_features]
y = df['target']  # y 값을 0과 1로 조정

# 결과를 저장할 딕셔너리 리스트 초기화
results = []

# tqdm을 사용하여 진행률 표시
for i in tqdm(range(1, 101), desc='Outer Loop (train_test_split random_state)'):
    for j in tqdm(range(1, 51), desc='Inner Loop (SVC random_state)', leave=False):
        # 데이터 분할
        X_train, X_test, y_train, y_test = train_test_split(X_top8, y, test_size=0.25, random_state=i)
        
        # SVC 모델 학습 및 예측
        svc_model = SVC(max_iter=10000, random_state=j)
        svc_model.fit(X_train, y_train)
        y_pred = svc_model.predict(X_test)
        
        # 정밀도 계산
        precision = precision_score(y_test, y_pred, average='macro', zero_division=0)
        
        # 결과 저장
        results.append({'Train_Test_Split_Random_State': i, 'SVC_Random_State': j, 'Precision': precision})

# 결과 리스트를 DataFrame으로 변환
results_df = pd.DataFrame(results)

# 정밀도가 가장 높은 상위 10개의 결과 선택
top_10_results = results_df.nlargest(10, 'Precision')

# 상위 10개 결과 출력
print(top_10_results)

Outer Loop (train_test_split random_state):   0%|          | 0/100 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

...

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

Inner Loop (SVC random_state):   0%|          | 0/50 [00:00<?, ?it/s]

     Train_Test_Split_Random_State  SVC_Random_State  Precision
650                             14                 1   0.662395
651                             14                 2   0.662395
652                             14                 3   0.662395
653                             14                 4   0.662395
654                             14                 5   0.662395
655                             14                 6   0.662395
656                             14                 7   0.662395
657                             14                 8   0.662395
658                             14                 9   0.662395
659                             14                10   0.662395
